# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import healpy as hp
import copy
import math
from math import exp, sqrt
import csv
import healpy as hp
import aipy as a
import scipy.constants
import scipy.integrate
import scipy.io
import pickle
import datetime
from multiprocessing import Pool
import time
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams["font.size"] = "18"
import warnings
warnings.filterwarnings("ignore")
from scipy.optimize import minimize

# Read sky maps

In [2]:
get_freq = lambda filename : int(filename.split('/')[-1].split('_')[1])
get_index = lambda x0: np.argmin(np.abs(freq_GHz-x0))

In [3]:
file_list = sorted(glob.glob('maps/*.txt'))
maps = {}

In [4]:
for ff in file_list:
    freq_map = get_freq(ff)
    if freq_map == 23000:
        freq_map = 22690
    _data = np.loadtxt(ff)
    maps[freq_map] = {} 
    maps[freq_map]['data'] = _data
    maps[freq_map]['npix'] = len(_data)
    maps[freq_map]['nside'] = hp.npix2nside(len(_data))    
maps = dict(sorted(maps.items()))

In [5]:
NSIDE = maps[22]['nside']
NPIXELS = hp.nside2npix(NSIDE)
NPARAM = 7 #parameters to optimize
NFREQ = len(list(maps.keys()))

In [6]:
list_of_freq = np.array(list(maps.keys()))
freq_GHz = list_of_freq/1e3
freq_GHz_indices = np.linspace(0, NFREQ, num=NFREQ, endpoint=False, dtype=int)

#Put maps in a 2D array such that each row is one pixel
maps_new = np.zeros((NPIXELS, NFREQ))

for ipixel in range(NPIXELS):
    all_temps = []
    for ff in list(maps.keys()):
        all_temps.append(maps[ff]['data'][ipixel])
    maps_new[ipixel]=np.array(all_temps)

# Function definitions and integrals

In [7]:
GSPAN0 = 100.0  # defines a factor by which the range of integration is expanded beyond required boundary
                # this is an initial guess and is updated for each sky pixel to avoid discontinuities in method

q_e = scipy.constants.elementary_charge
PI = scipy.constants.pi
Bmag = 1e-9  # Tesla == 10 micro-Gauss
sin_alph = 1.0
m_e = scipy.constants.m_e
cvel = scipy.constants.c
scale_gam_nu = (3.0*q_e*Bmag*sin_alph)/(4.0*PI*m_e*cvel)  ## when multiplied by gamma^2 gives critical freq


def modbessik2(u):  # Modified Bessel Fn of second kind 
                    # for non-negative real fractional order for real positive argument

    uu = max(0.01,u) # np.where ((u<0.01),0.01,u)
    bk = scipy.special.kve(5.0/3.0,1/uu,out=None)
    return (bk/(exp(1/uu)*uu*uu))

def fofx1float(gama,nu,scale_gam_nu,C1,fmult):  

    nu_c = ( (gama*gama) * (scale_gam_nu/1e9) )
    x = (nu/nu_c)
    rint =  scipy.integrate.quad(modbessik2,0.0,(1.0/x))             
    p1 = -((2*C1) - 3.0)
    return fmult*rint[0]*(gama**p1)*(x)

def integral_func1(p1,p2,a1,a2,a3,a4):
    return (scipy.integrate.quad(fofx1float,p1,p2,epsabs=1.0e-4/a4,epsrel=1e-08,\
                                 args=(a1,a2,a3,a4)))[0]

integ1 = np.vectorize(integral_func1,excluded=['p1','p2','a2','a3','a4'])
integ2 = np.vectorize(integral_func1,excluded=['p1','p2','a2','a3','a4'])

# Parameters

$C_1$, $\alpha_1$, $\delta_{\alpha}$, $\nu_{\rm br}$, $C_2$, $T_{e}$, $I_{x}$ and $\nu_{t}$

$\alpha_2 = \alpha_1 + \delta_{\alpha}$

Case for convex spectra ($\alpha_2 > \alpha_1$) 
$T(\nu) = C_{\rm 1}\Bigg(\nu^{-2}\Big\{\gamma_{\rm break}^{2\alpha_{\rm 1} -3}\int\limits_{\gamma_{\rm min}}^{\gamma_{\rm break}} F(x)~x~\gamma^{-(2\alpha_{\rm 1} -3)}\,d\gamma~+~\gamma_{\rm break}^{2\alpha_{\rm 2} -3}\int\limits_{\gamma_{\rm break}}^{\gamma_{\rm max}} F(x)~x~\gamma^{-(2\alpha_{\rm 2} -3)}\,d\gamma\Big\}~+
I_{\rm x}\nu^{-2.1}\Bigg)~e^{-(\frac{\nu_{\rm t}}{\nu})^{2.1}} + T_{\rm e}\Bigg(1 - e^{-(\frac{\nu_{\rm t}}{\nu})^{2.1}}\Bigg)$


Case for concave spectra ($\alpha_2 < \alpha_1$)\
$T(\nu) = C_1\Bigg(\nu^{-\alpha_{\rm 1}} + \frac{C_{\rm 2}}{C_{\rm 1}}\nu^{-\alpha_{\rm 2}} + I_{\rm x}~\nu^{-2.1}\Bigg){\rm e}^{-(\frac{\nu_{\rm t}}{\nu})^{2.1}} + T_{\rm e}\Bigg(1 - e^{-(\frac{\nu_{\rm t}}{\nu})^{2.1}}\Bigg)$

# Intitial Guess Generation

In [8]:
alpha_1_array = (np.log10(maps_new[:,get_index(45/1e3)]) - np.log10(maps_new[:,get_index(150/1e3)]))/\
(np.log10(150/1e3)-np.log10(45/1e3))
alpha_2_array = (np.log10(maps_new[:,get_index(408/1e3)]) - np.log10(maps_new[:,get_index(1420/1e3)]))/\
(np.log10(1420/1e3)-np.log10(408/1e3))

#Set all values to between 2 and 3
ind_alpha1_array_less_2 = (alpha_1_array<2.0)
alpha_1_array[ind_alpha1_array_less_2] = 2.0

ind_alpha1_array_more_3 = (alpha_1_array>3.0)
alpha_1_array[ind_alpha1_array_more_3] = 3.0

ind_alpha2_array_less_2 = (alpha_2_array<2.0)
alpha_2_array[ind_alpha2_array_less_2] = 2.0

ind_alpha2_array_more_3 = (alpha_2_array>3.0)
alpha_2_array[ind_alpha2_array_more_3] = 3.0

dalpha_array = alpha_2_array - alpha_1_array

T_e_array = 8000*np.ones(NPIXELS)
nu_t_array = 1/1e3 * np.ones(NPIXELS) #GHz
nu_break_array = sqrt(150*408)*1e6 *np.ones(NPIXELS) #Hz

Above takes care of $\alpha$, $\delta_a$, $T_e$, $\nu_t$, $\nu_{\rm br}$

Next we look at $I_x$, $C_1$, $C_2$

In [9]:
#Note: All parameters are in linear units (no log10 at this stage)
flow_GHz, fhigh_GHz = 1.420, 22.690
index_flow_GHz = get_index(flow_GHz)
index_fhigh_GHz = get_index(fhigh_GHz)

concave_low_GHz = 150./1e3
concave_high_GHz = 408./1e3

norm1_array = np.zeros(NPIXELS)
norm2_array = np.zeros(NPIXELS)
Tx_array = np.zeros(NPIXELS)
flag_array = np.zeros(NPIXELS)

for ipixel in range(NPIXELS):
    
    print(f"Processing pixel {ipixel}...")
    
    alpha1 = alpha_1_array[ipixel]
    alpha2 = alpha1 + dalpha_array[ipixel]
    
    gama_break = np.sqrt((nu_break_array[ipixel])/scale_gam_nu)

    if alpha2 > alpha1: # convex spectra
        
        flag_array[ipixel] = 0
        
        #Compute normalization at f = 1.420 GHz
        
        nu_min = flow_GHz*1e9/GSPAN0
        nu_max = flow_GHz*1e9*GSPAN0
        
        xl = np.sqrt((nu_min)/scale_gam_nu)
        xu = np.sqrt((nu_max)/scale_gam_nu)
        xb = gama_break
        
        extn = np.exp(-1.0*((nu_t_array[ipixel]/flow_GHz)**2.1))

        if xl > xb or xu < xb: #break is outside band of interest

            if xl > xb: 
                C1 = alpha2
            else: 
                C1 = alpha1
    
            fmult =  ((gama_break)**(2*C1-3))    

            cspectT = (scipy.integrate.quad(fofx1float,xl,xu,\
                                      args=(flow_GHz,scale_gam_nu,C1,fmult)))[0]

        else:

            xl1 = xl
            xu1 = xb
            fmult1 = (gama_break) ** (2 * alpha1 - 3)

            xl2 = xb
            xu2 = xu
            fmult2 = (gama_break) ** (2 * alpha2 - 3)

            cspectT1 = integ1(xl1, xu1, flow_GHz, scale_gam_nu, alpha1, fmult1)
            cspectT2 = integ1(xl2, xu2, flow_GHz, scale_gam_nu, alpha2, fmult2)

            cspectT = cspectT1 + cspectT2
            
        fnorm = (maps_new[ipixel][index_flow_GHz] - T_e_array[ipixel]*(1.0-extn))/\
        (flow_GHz**(-2)*cspectT*extn)

        norm1_array[ipixel] = fnorm
        
        #Apply the above normalization at f = 22.690 GHz
        
        nu_min = fhigh_GHz*1e9/GSPAN0
        nu_max = fhigh_GHz*1e9*GSPAN0
        
        xl = np.sqrt((nu_min)/scale_gam_nu)
        xu = np.sqrt((nu_max)/scale_gam_nu)
        xb = gama_break
        
        extn = np.exp(-1.0*((nu_t_array[ipixel]/fhigh_GHz)**2.1))

        if xl > xb or xu < xb: #break is outside band of interest

            if xl > xb: 
                C1 = alpha2
            else: 
                C1 = alpha1
    
            fmult =  ((gama_break)**(2*C1-3))    

            cspectT = (scipy.integrate.quad(fofx1float,xl,xu,\
                                      args=(fhigh_GHz,scale_gam_nu,C1,fmult)))[0]

        else:

            xl1 = xl
            xu1 = xb
            fmult1 = (gama_break) ** (2 * alpha1 - 3)

            xl2 = xb
            xu2 = xu
            fmult2 = (gama_break) ** (2 * alpha2 - 3)

            cspectT1 = integ1(xl1, xu1, fhigh_GHz, scale_gam_nu, alpha1, fmult1)
            cspectT2 = integ1(xl2, xu2, fhigh_GHz, scale_gam_nu, alpha2, fmult2)

            cspectT = cspectT1 + cspectT2
            
        Tx_num_term1 = (maps_new[ipixel][index_fhigh_GHz] - T_e_array[ipixel]*(1.0-extn))
        Tx_num_term2 = fhigh_GHz**(-2)*cspectT
        Tx_num = Tx_num_term1 - Tx_num_term2
        Tx = Tx_num/flow_GHz**(-2.1) #not quite sure why we are using 1.4 GHz here, instead of 23 GHz
        
        if Tx <= 0.0:
            Tx = 1e-12
            
        Tx_array[ipixel] = Tx
        
    elif alpha1 >= alpha2:
        
        flag_array[ipixel] = 1
        
        norm1_array[ipixel] = maps_new[ipixel][get_index(concave_low_GHz)]/\
        (concave_low_GHz**(-alpha1))
        
        norm2_array[ipixel] = maps_new[ipixel][get_index(concave_high_GHz)]/\
        (concave_high_GHz**(-alpha2))/norm1_array[ipixel] #this is C2/C1
        
        nu_break_array[ipixel] = norm2_array[ipixel]
        
        extn = np.exp(-1.0*((nu_t_array[ipixel]/fhigh_GHz)**2.1))
        
        Tx_term1 = (maps_new[ipixel][get_index(fhigh_GHz)] - T_e_array[ipixel]*(1-extn))/\
        (norm1_array[ipixel]*extn)
        
        Tx_term2 = fhigh_GHz**(-alpha1) + norm2_array[ipixel]*fhigh_GHz**(-alpha2)
        
        Tx = ((Tx_term1) - (Tx_term2))/(fhigh_GHz**(-2.1))
        
        if Tx <= 0.0:
            Tx = 1e-12
        
        Tx_array[ipixel] = Tx

Processing pixel 0...
Processing pixel 1...
Processing pixel 2...
Processing pixel 3...
Processing pixel 4...
Processing pixel 5...
Processing pixel 6...
Processing pixel 7...
Processing pixel 8...
Processing pixel 9...
Processing pixel 10...
Processing pixel 11...
Processing pixel 12...
Processing pixel 13...
Processing pixel 14...
Processing pixel 15...
Processing pixel 16...
Processing pixel 17...
Processing pixel 18...
Processing pixel 19...
Processing pixel 20...
Processing pixel 21...
Processing pixel 22...
Processing pixel 23...
Processing pixel 24...
Processing pixel 25...
Processing pixel 26...
Processing pixel 27...
Processing pixel 28...
Processing pixel 29...
Processing pixel 30...
Processing pixel 31...
Processing pixel 32...
Processing pixel 33...
Processing pixel 34...
Processing pixel 35...
Processing pixel 36...
Processing pixel 37...
Processing pixel 38...
Processing pixel 39...
Processing pixel 40...
Processing pixel 41...
Processing pixel 42...
Processing pixel 43..

Processing pixel 1080...
Processing pixel 1081...
Processing pixel 1082...
Processing pixel 1083...
Processing pixel 1084...
Processing pixel 1085...
Processing pixel 1086...
Processing pixel 1087...
Processing pixel 1088...
Processing pixel 1089...
Processing pixel 1090...
Processing pixel 1091...
Processing pixel 1092...
Processing pixel 1093...
Processing pixel 1094...
Processing pixel 1095...
Processing pixel 1096...
Processing pixel 1097...
Processing pixel 1098...
Processing pixel 1099...
Processing pixel 1100...
Processing pixel 1101...
Processing pixel 1102...
Processing pixel 1103...
Processing pixel 1104...
Processing pixel 1105...
Processing pixel 1106...
Processing pixel 1107...
Processing pixel 1108...
Processing pixel 1109...
Processing pixel 1110...
Processing pixel 1111...
Processing pixel 1112...
Processing pixel 1113...
Processing pixel 1114...
Processing pixel 1115...
Processing pixel 1116...
Processing pixel 1117...
Processing pixel 1118...
Processing pixel 1119...


# Optimization

In [10]:
# A function that may be called by an optimisation code: 
# pyGMOSS_Func_ComputeVariance_v0(x, flag_param, freq_array, Tb_array)
#
# This function receives a set of variable and static parameters
#        that determine the phyiscal spectrum at each sky pixel.
# And arrays of frequency and brightness.
# Returns a variance measure of deviation between computed spectral values and data
#
# The call to scipy.optimize.minimize would contain
# Initial guess for the variable parameters
# x is a 1-D array with shape (n=7) containing, in this order, the variable parameters:
    # norm_param
    # alpha1_param
    # dalpha_param
    # nubrk_param
    # Tx_param
    # Te_param
    # nut_param
#
# args in the call to the optimisation would be passed to this function, and
# should be a tuple of the fixed parameter flag and arrays containing frequencies and brightness temperatures.
    #
    # flag_param: 0 where alpha2 steeper than alpha1 (convex spectrum)
    #             1 where data requires alpha2 flatter than alpha1 (concave spectrum)
    #             alpha1, alpha2 are defined with convention: T(\nu) proportional to \nu^(-alpha)
    #             alpha2 is the spectral index at higher frequencies, alpha1 at lower frequencies
    #             when concave then model as sum of power laws.
    #             when convex then model as SSA and/or FF absorption to give low frequency turnover.
    #
    # freq_array: float array of frequencies (MHz) at which brightness data is available for current pixel.
    #
    # Tb_array: float array of kelvin brightness temperatures at the frequencies listed.

In [11]:
def pyGMOSS_Func_ComputeVariance(x, flag_param, freq_GHz, Tb_array):
    
    norm_param = x[0]
    alpha1_param = x[1]
    dalpha_param = x[2]
    nubrk_param = x[3]
    Tx_param = x[4]
    Te_param = x[5]
    nut_param = x[6]
    
    flag = flag_param
    
    Tx = 10.0**Tx_param
    Te = 10.0**Te_param
    nu_t = 10.0**nut_param
    alpha1 = 10.0**alpha1_param
    
    extn = np.exp(-1.0*((nu_t/freq_GHz)**2.1))
    
    if flag == 0:  ## the case where alpha2 steeper than alpha1

        fnorm = 10.0**norm_param
        alpha2 = alpha1 + 10.0**dalpha_param
        nu_break = 10.0**nubrk_param
        gama_break = np.sqrt((nu_break)/scale_gam_nu)
        xb = gama_break
        
        nu_min0 = freq_GHz[0]*1e9/GSPAN0
        nu_max0 = freq_GHz[-1]*1e9*GSPAN0
        xl0 = np.sqrt((nu_min0)/scale_gam_nu)
        xu0 = np.sqrt((nu_max0)/scale_gam_nu)
        
        if xl0 > xb or xu0 < xb:
            
            if xl0 > xb: 
                C1 = alpha2
            else: 
                C1 = alpha1
            fmult =  ((gama_break)**(2*C1-3))    
                
            xl = np.sqrt((freq_GHz*1e9)/(GSPAN0*scale_gam_nu))
            xu = np.sqrt((freq_GHz*1e9*GSPAN0)/scale_gam_nu)
            
            index_array = np.transpose([xl,xu,freq_GHz])
    
            cspectT = [(scipy.integrate.quad(fofx1float,xl[i],xu[i],\
                                      args=(freq_GHz[i],scale_gam_nu,C1,fmult)))[0] for i in freq_GHz_indices]
            cspect = fnorm*((freq_GHz**-2.0) * cspectT + Tx*(freq_GHz**-2.1))*extn + Te*(1.0-extn)

                                 
        else:
            
            xl1 = xl0
            xu1 = xb
            fmult1 = (gama_break)**(2*alpha1-3)
            xl2 = xb
            xu2 = xu0
            fmult2 = ((gama_break)**(2*alpha2-3))
                        
            cspectT1 = integ1(xl1,xu1,freq_GHz,scale_gam_nu,alpha1,fmult1)
            cspectT2 = integ1(xl2,xu2,freq_GHz,scale_gam_nu,alpha2,fmult2)
            
            cspect = fnorm * ((freq_GHz**-2.0)*np.add(cspectT1,cspectT2)+Tx*(freq_GHz**-2.1))*extn\
                                + Te*(1.0-extn)
            
    else:            ##   If data requires alpha2 flatter than alpha1, 
                     ##   then model as sum of power laws, i.e. steep and flat spectrum sources
            
        fnorm1 = 10**norm_param
        fnorm2 = 10**nubrk_param
        alpha2 = alpha1 - 10.0**dalpha_param
        
        cspect = fnorm1*( (freq_GHz**(-alpha1)) + fnorm2*(freq_GHz**(-alpha2)) + \
                    Tx*(freq_GHz**-2.1) )*extn + Te*(1.0 - extn)

    var_ModelData = np.sum((cspect-Tb_array)**2)/len(freq_GHz)
    
    #Check for bounds for certain parameters:
    
    return var_ModelData

In [12]:
optimizations = {}
coefficient_array = np.zeros((NPIXELS,NPARAM))

for ipixel in range(NPIXELS):
    print(f"Processing pixel: {ipixel}...")
    
    optimizations[ipixel] = {}
    
    x = np.zeros(NPARAM)
    
    x[0], x[1], x[2], x[3], x[4], x[5], x[6] = np.log10(norm1_array[ipixel]), np.log10(alpha_1_array[ipixel]),\
    np.log10(dalpha_array[ipixel]), np.log10(nu_break_array[ipixel]), \
    np.log10(Tx_array[ipixel]), np.log10(T_e_array[ipixel]), np.log10(nu_t_array[ipixel])
    
    res = minimize(pyGMOSS_Func_ComputeVariance, x, args=(flag_array[ipixel], freq_GHz, maps_new[ipixel]))
    
    optimizations[ipixel]['pixel'] = ipixel
    optimizations[ipixel]['coefficients'] = res.x
    optimizations[ipixel]['success'] = res.success
    optimizations[ipixel]['initial_guess'] = x
    optimizations[ipixel]['flag'] = flag_array[ipixel]

Processing pixel: 0...
Processing pixel: 1...
Processing pixel: 2...
Processing pixel: 3...
Processing pixel: 4...
Processing pixel: 5...
Processing pixel: 6...
Processing pixel: 7...
Processing pixel: 8...
Processing pixel: 9...
Processing pixel: 10...
Processing pixel: 11...
Processing pixel: 12...
Processing pixel: 13...
Processing pixel: 14...
Processing pixel: 15...
Processing pixel: 16...
Processing pixel: 17...
Processing pixel: 18...
Processing pixel: 19...
Processing pixel: 20...
Processing pixel: 21...
Processing pixel: 22...
Processing pixel: 23...
Processing pixel: 24...
Processing pixel: 25...
Processing pixel: 26...
Processing pixel: 27...
Processing pixel: 28...
Processing pixel: 29...
Processing pixel: 30...
Processing pixel: 31...
Processing pixel: 32...
Processing pixel: 33...
Processing pixel: 34...
Processing pixel: 35...
Processing pixel: 36...
Processing pixel: 37...
Processing pixel: 38...
Processing pixel: 39...
Processing pixel: 40...
Processing pixel: 41...
Pr

Processing pixel: 355...
Processing pixel: 356...
Processing pixel: 357...
Processing pixel: 358...
Processing pixel: 359...
Processing pixel: 360...
Processing pixel: 361...
Processing pixel: 362...
Processing pixel: 363...
Processing pixel: 364...
Processing pixel: 365...
Processing pixel: 366...
Processing pixel: 367...
Processing pixel: 368...
Processing pixel: 369...
Processing pixel: 370...
Processing pixel: 371...
Processing pixel: 372...
Processing pixel: 373...
Processing pixel: 374...
Processing pixel: 375...
Processing pixel: 376...
Processing pixel: 377...
Processing pixel: 378...
Processing pixel: 379...
Processing pixel: 380...
Processing pixel: 381...
Processing pixel: 382...
Processing pixel: 383...
Processing pixel: 384...
Processing pixel: 385...
Processing pixel: 386...
Processing pixel: 387...
Processing pixel: 388...
Processing pixel: 389...
Processing pixel: 390...
Processing pixel: 391...
Processing pixel: 392...
Processing pixel: 393...
Processing pixel: 394...


Processing pixel: 831...
Processing pixel: 832...
Processing pixel: 833...
Processing pixel: 834...
Processing pixel: 835...
Processing pixel: 836...
Processing pixel: 837...
Processing pixel: 838...
Processing pixel: 839...
Processing pixel: 840...
Processing pixel: 841...
Processing pixel: 842...
Processing pixel: 843...
Processing pixel: 844...
Processing pixel: 845...
Processing pixel: 846...
Processing pixel: 847...
Processing pixel: 848...
Processing pixel: 849...
Processing pixel: 850...
Processing pixel: 851...
Processing pixel: 852...
Processing pixel: 853...
Processing pixel: 854...
Processing pixel: 855...
Processing pixel: 856...
Processing pixel: 857...
Processing pixel: 858...
Processing pixel: 859...
Processing pixel: 860...
Processing pixel: 861...
Processing pixel: 862...
Processing pixel: 863...
Processing pixel: 864...
Processing pixel: 865...
Processing pixel: 866...
Processing pixel: 867...
Processing pixel: 868...
Processing pixel: 869...
Processing pixel: 870...


Processing pixel: 1167...
Processing pixel: 1168...
Processing pixel: 1169...
Processing pixel: 1170...
Processing pixel: 1171...
Processing pixel: 1172...
Processing pixel: 1173...
Processing pixel: 1174...
Processing pixel: 1175...
Processing pixel: 1176...
Processing pixel: 1177...
Processing pixel: 1178...
Processing pixel: 1179...
Processing pixel: 1180...
Processing pixel: 1181...
Processing pixel: 1182...
Processing pixel: 1183...
Processing pixel: 1184...
Processing pixel: 1185...
Processing pixel: 1186...
Processing pixel: 1187...
Processing pixel: 1188...
Processing pixel: 1189...
Processing pixel: 1190...
Processing pixel: 1191...
Processing pixel: 1192...
Processing pixel: 1193...
Processing pixel: 1194...
Processing pixel: 1195...
Processing pixel: 1196...
Processing pixel: 1197...
Processing pixel: 1198...
Processing pixel: 1199...
Processing pixel: 1200...
Processing pixel: 1201...
Processing pixel: 1202...
Processing pixel: 1203...
Processing pixel: 1204...
Processing p

Processing pixel: 1703...
Processing pixel: 1704...
Processing pixel: 1705...
Processing pixel: 1706...
Processing pixel: 1707...
Processing pixel: 1708...
Processing pixel: 1709...
Processing pixel: 1710...
Processing pixel: 1711...
Processing pixel: 1712...
Processing pixel: 1713...
Processing pixel: 1714...
Processing pixel: 1715...
Processing pixel: 1716...
Processing pixel: 1717...
Processing pixel: 1718...
Processing pixel: 1719...
Processing pixel: 1720...
Processing pixel: 1721...
Processing pixel: 1722...
Processing pixel: 1723...
Processing pixel: 1724...
Processing pixel: 1725...
Processing pixel: 1726...
Processing pixel: 1727...
Processing pixel: 1728...
Processing pixel: 1729...
Processing pixel: 1730...
Processing pixel: 1731...
Processing pixel: 1732...
Processing pixel: 1733...
Processing pixel: 1734...
Processing pixel: 1735...
Processing pixel: 1736...
Processing pixel: 1737...
Processing pixel: 1738...
Processing pixel: 1739...
Processing pixel: 1740...
Processing p

Processing pixel: 2082...
Processing pixel: 2083...
Processing pixel: 2084...
Processing pixel: 2085...
Processing pixel: 2086...
Processing pixel: 2087...
Processing pixel: 2088...
Processing pixel: 2089...
Processing pixel: 2090...
Processing pixel: 2091...
Processing pixel: 2092...
Processing pixel: 2093...
Processing pixel: 2094...
Processing pixel: 2095...
Processing pixel: 2096...
Processing pixel: 2097...
Processing pixel: 2098...
Processing pixel: 2099...
Processing pixel: 2100...
Processing pixel: 2101...
Processing pixel: 2102...
Processing pixel: 2103...
Processing pixel: 2104...
Processing pixel: 2105...
Processing pixel: 2106...
Processing pixel: 2107...
Processing pixel: 2108...
Processing pixel: 2109...
Processing pixel: 2110...
Processing pixel: 2111...
Processing pixel: 2112...
Processing pixel: 2113...
Processing pixel: 2114...
Processing pixel: 2115...
Processing pixel: 2116...
Processing pixel: 2117...
Processing pixel: 2118...
Processing pixel: 2119...
Processing p

Processing pixel: 2556...
Processing pixel: 2557...
Processing pixel: 2558...
Processing pixel: 2559...
Processing pixel: 2560...
Processing pixel: 2561...
Processing pixel: 2562...
Processing pixel: 2563...
Processing pixel: 2564...
Processing pixel: 2565...
Processing pixel: 2566...
Processing pixel: 2567...
Processing pixel: 2568...
Processing pixel: 2569...
Processing pixel: 2570...
Processing pixel: 2571...
Processing pixel: 2572...
Processing pixel: 2573...
Processing pixel: 2574...
Processing pixel: 2575...
Processing pixel: 2576...
Processing pixel: 2577...
Processing pixel: 2578...
Processing pixel: 2579...
Processing pixel: 2580...
Processing pixel: 2581...
Processing pixel: 2582...
Processing pixel: 2583...
Processing pixel: 2584...
Processing pixel: 2585...
Processing pixel: 2586...
Processing pixel: 2587...
Processing pixel: 2588...
Processing pixel: 2589...
Processing pixel: 2590...
Processing pixel: 2591...
Processing pixel: 2592...
Processing pixel: 2593...
Processing p

Processing pixel: 3037...
Processing pixel: 3038...
Processing pixel: 3039...
Processing pixel: 3040...
Processing pixel: 3041...
Processing pixel: 3042...
Processing pixel: 3043...
Processing pixel: 3044...
Processing pixel: 3045...
Processing pixel: 3046...
Processing pixel: 3047...
Processing pixel: 3048...
Processing pixel: 3049...
Processing pixel: 3050...
Processing pixel: 3051...
Processing pixel: 3052...
Processing pixel: 3053...
Processing pixel: 3054...
Processing pixel: 3055...
Processing pixel: 3056...
Processing pixel: 3057...
Processing pixel: 3058...
Processing pixel: 3059...
Processing pixel: 3060...
Processing pixel: 3061...
Processing pixel: 3062...
Processing pixel: 3063...
Processing pixel: 3064...
Processing pixel: 3065...
Processing pixel: 3066...
Processing pixel: 3067...
Processing pixel: 3068...
Processing pixel: 3069...
Processing pixel: 3070...
Processing pixel: 3071...
